In [80]:
import os
import re
import uuid
from docx import Document
from docx.shared import Pt
import win32com.client
from pymongo import MongoClient
import pyodbc
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

# ---------- הגדרות ----------
mongo_client = MongoClient("mongodb://localhost:27017")
mongo_db = mongo_client["Manage_Files_DB"]
values_collection = mongo_db["Customers_Fields"]

# התחברות ל-SQL Server (התאימי לפרטים שלך)
def get_db_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-C0CP90K\\MSSQLSERVER01;"
        "DATABASE=ManageDB;"
        "Trusted_Connection=yes;"
    )

VARIABLE_REGEX = r"\{\{(.*?)\}\}"

# ---------- פונקציות ----------

def convert_doc_to_docx(input_path):
    try:
        word = win32com.client.Dispatch("Word.Application")
        doc = word.Documents.Open(input_path)
        output_path = os.path.splitext(input_path)[0] + ".docx"
        doc.SaveAs(output_path, FileFormat=16)  # 16 is the code for docx format
        doc.Close()
        word.Quit()
        return output_path
    except Exception as e:
        print("שגיאה בהמרה ל-docx:", e)
        return None

def create_highlighted_run(text, paragraph):
    run = paragraph.add_run(text)
    run.font.highlight_color = 7  # צהוב
    run.font.name = 'Arial'
    run.font.size = Pt(12)
    return run

def process_docx(path):
    doc = Document(path)
    conn = get_db_connection()
    cursor = conn.cursor()
    variables_not_found = []

    for para in doc.paragraphs:
        if not re.search(VARIABLE_REGEX, para.text):
            continue

        new_text = ""
        last_end = 0

        # עובר על כל המשתנים בתוך הטקסט
        for match in re.finditer(VARIABLE_REGEX, para.text):
            full_match = match.group(0)
            start, end = match.span()

            new_text += para.text[last_end:start]

            parts = match.group(1).split(',')
            if len(parts) < 5:
                new_text += full_match
                last_end = end
                continue

            var_name, var_type, var_len, var_required, var_description = [p.strip() for p in parts]

            # SQL: בדיקה אם קיים
            cursor.execute("SELECT COUNT(*) FROM Fields WHERE name = ?", (var_name,))
            if cursor.fetchone()[0] == 0:
                cursor.execute("INSERT INTO Fields (name, type, length, required) VALUES (?, ?, ?, ?)",
                               (var_name, var_type, int(var_len), int(var_required == '*')))
                conn.commit()

            # MongoDB
            value_doc = values_collection.find_one({"name": var_name})
            value = value_doc.get("value") if value_doc else None

            if not value:
                variables_not_found.append(var_name)
                value = "     "  # 5 רווחים

                # הוספת 5 רווחים מודגשים בצהוב
                create_highlighted_run("     ", para)

            new_text += str(value)
            last_end = end

        new_text += para.text[last_end:]
        para.clear()  # לא לשכוח לנקות את הפסקה לפני הוספת הטקסט החדש
        run = para.add_run(new_text)
        run.font.name = 'Arial'
        run.font.size = Pt(12)

    # שמירה עם שם + טופל
    dir_path = os.path.dirname(path)
    filename_wo_ext = os.path.splitext(os.path.basename(path))[0]
    output_path = os.path.join(dir_path, f"{filename_wo_ext} טופל.docx")
    doc.save(output_path)
    print("✅ קובץ נשמר כ:", output_path)
    if variables_not_found:
        print("⚠️ משתנים שלא נמצאו:", variables_not_found)

# ---------- הרצה ----------

# נתיב לקובץ Word לבדיקה (doc או docx)
input_path = "C:\\Users\\User\\Desktop\\מסמכים לפרויקט\\ממופה\\יפוי כוח.doc"  # שנה לפי הצורך

if input_path.endswith(".doc"):
    input_path = convert_doc_to_docx(input_path)

process_docx(input_path)


✅ קובץ נשמר כ: C:\Users\User\Desktop\מסמכים לפרויקט\ממופה\יפוי כוח טופל.docx
⚠️ משתנים שלא נמצאו: ['buyer_first_name', 'buyer_last_name', 'buyer_id', 'property_address_street', 'property_address_number', 'property_address_city', 'property_block_number', 'property_parcel_number', 'property_subparcel_numbers', 'seller_first_name', 'seller_last_name', 'seller_id']


In [81]:
import os
import re
import uuid
from docx import Document
from docx.shared import Pt
import win32com.client
from pymongo import MongoClient
import pyodbc
from docx.enum.text import WD_COLOR_INDEX

# ---------- הגדרות ----------
mongo_client = MongoClient("mongodb://localhost:27017")
mongo_db = mongo_client["Manage_Files_DB"]
values_collection = mongo_db["Customers_Fields"]

# התחברות ל-SQL Server
def get_db_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-C0CP90K\\MSSQLSERVER01;"
        "DATABASE=ManageDB;"
        "Trusted_Connection=yes;"
    )

VARIABLE_REGEX = r"\{\{(.*?)\}\}"

# ---------- פונקציות ----------

def convert_doc_to_docx(input_path):
    try:
        word = win32com.client.Dispatch("Word.Application")
        doc = word.Documents.Open(input_path)
        output_path = os.path.splitext(input_path)[0] + ".docx"
        doc.SaveAs(output_path, FileFormat=16)  # 16 is docx format
        doc.Close()
        word.Quit()
        return output_path
    except Exception as e:
        print("שגיאה בהמרה ל-docx:", e)
        return None

def process_paragraphs(paragraphs, cursor, variables_not_found,sql_id):
    for para in paragraphs:
        matches = list(re.finditer(VARIABLE_REGEX, para.text))
        if not matches:
            continue

        original_text = para.text
        para.clear()
        last_end = 0

        for match in matches:
            start, end = match.span()
            var_text = match.group(1)
            parts = var_text.split(',')

            before = original_text[last_end:start]
            if before:
                run = para.add_run(before)
                run.font.name = 'Arial'
                run.font.size = Pt(12)

            if len(parts) < 5:
                run = para.add_run(match.group(0))
                run.font.name = 'Arial'
                run.font.size = Pt(12)
                last_end = end
                continue

            var_name, var_type, var_len, var_required, var_description = [p.strip() for p in parts]

            # בדיקת קיום ב-SQL
            cursor.execute("SELECT COUNT(*) FROM Fields WHERE name = ?", (var_name,))
            if cursor.fetchone()[0] == 0:
                cursor.execute("INSERT INTO Fields (name, type, length, required) VALUES (?, ?, ?, ?)",
                               (var_name, var_type, int(var_len), int(var_required == '*')))
                cursor.connection.commit()

            # שליפת ערך מ-MongoDB
            value_doc = values_collection.find_one({"name": var_name})
            value = value_doc.get("value") if value_doc else None

            if value:
                run = para.add_run(str(value))
                run.font.name = 'Arial'
                run.font.size = Pt(12)
            else:
                variables_not_found.append(var_name)
                run = para.add_run("-----")
                run.font.highlight_color = WD_COLOR_INDEX.YELLOW
                run.font.name = 'Arial'
                run.font.size = Pt(12)

            last_end = end

        after = original_text[last_end:]
        if after:
            run = para.add_run(after)
            run.font.name = 'Arial'
            run.font.size = Pt(12)

def process_docx(path,sql_id):
    doc = Document(path)
    conn = get_db_connection()
    cursor = conn.cursor()
    variables_not_found = []

    # פסקאות בגוף
    process_paragraphs(doc.paragraphs, cursor, variables_not_found,sql_id)

    # טבלאות
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                process_paragraphs(cell.paragraphs, cursor, variables_not_found,sql_id)

    # כותרות עליונות ותחתונות
    for section in doc.sections:
        process_paragraphs(section.header.paragraphs, cursor, variables_not_found,sql_id)
        process_paragraphs(section.footer.paragraphs, cursor, variables_not_found,sql_id)

    # שמירה
    dir_path = os.path.dirname(path)
    filename_wo_ext = os.path.splitext(os.path.basename(path))[0]
    output_path = os.path.join(dir_path, f"{filename_wo_ext} טופל.docx")
    doc.save(output_path)

    print("✅ קובץ נשמר כ:", output_path)
    if variables_not_found:
        print("⚠️ משתנים שלא נמצאו:", variables_not_found)

# ---------- הרצה ----------

# נתיב לקובץ Word לבדיקה (doc או docx)
input_path = "C:\\Users\\User\\Desktop\\מסמכים לפרויקט\\ממופה\\יפוי כוח.doc"

if input_path.endswith(".doc"):
    input_path = convert_doc_to_docx(input_path)

process_docx(input_path,"3")


✅ קובץ נשמר כ: C:\Users\User\Desktop\מסמכים לפרויקט\ממופה\יפוי כוח טופל.docx
⚠️ משתנים שלא נמצאו: ['buyer_first_name', 'buyer_last_name', 'buyer_id', 'property_address_street', 'property_address_number', 'property_address_city', 'property_block_number', 'property_parcel_number', 'property_subparcel_numbers', 'seller_first_name', 'seller_last_name', 'seller_id']


In [62]:
import re

VARIABLE_REGEX = r"\{\{([a-zA-Z0-9_]+(?:,[a-zA-Z0-9_]+)*)\}\}"
test = "שלום {{customer_name,text,100,*,שם}} מה שלומך?"
print(re.findall(VARIABLE_REGEX, test))

[]


In [73]:
import re
from datetime import datetime

def validate_value_by_type(value, sql_type, field_name=None, errors=None):
    """
    מחזירה את הערך המומר לפי סוג השדה. במידה ולא ניתן להמיר – תזרוק שגיאה או תוסיף לרשימת השגיאות.
    :param value: הערך לבדיקה
    :param sql_type: סוג ה-SQL (כמו varchar(50), int, וכו')
    :param field_name: שם השדה (רק לצורכי שגיאה)
    :param errors: רשימת שגיאות למילוי (אם קיימת)
    :return: הערך המתוקן
    """
    type_match = re.match(r"([a-zA-Z]+)\(?(\d*)\)?", sql_type.strip())
    if not type_match:
        return value

    base_type = type_match.group(1).lower()
    length_str = type_match.group(2)
    max_length = int(length_str) if length_str.isdigit() else None

    try:
        if base_type in ['varchar', 'char', 'nchar', 'nvarchar']:
            str_value = str(value)
            if max_length and len(str_value) > max_length:
                return str_value[:max_length]
            return str_value

        elif base_type in ['int', 'bigint', 'smallint']:
            return int(value)

        elif base_type in ['float', 'real', 'decimal', 'numeric']:
            return float(value)

        elif base_type in ['bit']:
            return bool(int(value))

        elif base_type in ['date', 'datetime', 'smalldatetime']:
            return datetime.fromisoformat(str(value))

        else:
            return str(value)

    except Exception as e:
        msg = f"שגיאה בשדה '{field_name or 'שדה לא מזוהה'}': הערך '{value}' לא תואם לסוג '{sql_type}'"
        if errors is not None:
            errors.append(msg)
            return value
        else:
            raise ValueError(msg)
errors = []
value = validate_value_by_type("abc", "int", field_name="buyer_id", errors=errors)

if errors:
    print("שגיאות שנמצאו:")
    for err in errors:
        print(err)


שגיאות שנמצאו:
שגיאה בשדה 'buyer_id': הערך 'abc' לא תואם לסוג 'int'


In [86]:
import os
import re
import uuid
from docx import Document
from docx.shared import Pt
import win32com.client
from pymongo import MongoClient
import pyodbc
from docx.enum.text import WD_COLOR_INDEX

# ---------- הגדרות ----------
mongo_client = MongoClient("mongodb://localhost:27017")
mongo_db = mongo_client["Manage_Files_DB"]
values_collection = mongo_db["Customers_Fields"]

def get_db_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-C0CP90K\\MSSQLSERVER01;"
        "DATABASE=ManageDB;"
        "Trusted_Connection=yes;"
    )
print("11111")
VARIABLE_REGEX = r"\{\{(.*?)\}\}"
print("22222")
def convert_doc_to_docx(input_path):
    try:
        word = win32com.client.Dispatch("Word.Application")
        doc = word.Documents.Open(input_path)
        output_path = os.path.splitext(input_path)[0] + ".docx"
        doc.SaveAs(output_path, FileFormat=16)  # 16 is docx format
        doc.Close()
        word.Quit()
        return output_path
    except Exception as e:
        print("שגיאה בהמרה ל-docx:", e)
        return None
print("33333")
def process_paragraphs(paragraphs, cursor, mongo_data, counters, variables_not_found,sql_id):
    print("44444")
    for para in paragraphs:
        matches = list(re.finditer(VARIABLE_REGEX, para.text))
        if not matches:
            continue

        original_text = para.text
        para.clear()
        last_end = 0

        for match in matches:
            start, end = match.span()
            var_text = match.group(1)
            parts = var_text.split(',')

            before = original_text[last_end:start]
            if before:
                run = para.add_run(before)
                run.font.name = 'Arial'
                run.font.size = Pt(12)

            if len(parts) < 5:
                run = para.add_run(match.group(0))
                run.font.name = 'Arial'
                run.font.size = Pt(12)
                last_end = end
                continue

            var_name, var_type, var_len, var_required, var_description = [p.strip() for p in parts]

            # בדיקת קיום ב-SQL Server
            cursor.execute("SELECT COUNT(*) FROM Fields WHERE name = ?", (var_name,))
            if cursor.fetchone()[0] == 0:
                cursor.execute(
                    "INSERT INTO Fields (sql_id, type, length, required, description) VALUES (?, ?, ?, ?, ?)",
                    (sql_id, var_type, int(var_len), int(var_required == '*'), var_description)
                )
                cursor.connection.commit()

            # שליפת ערך לפי sql_id
            value = mongo_data.get(sql_id)

            if value:
                count = counters.get(sql_id, 0)
                if isinstance(value, list):
                    if count < len(value):
                        final_value = value[count]
                        counters[sql_id] = count + 1
                    else:
                        final_value = ""
                else:
                    final_value = value

                run = para.add_run(str(final_value))
                run.font.name = 'Arial'
                run.font.size = Pt(12)
            else:
                variables_not_found.append(sql_id)
                run = para.add_run("-----")
                run.font.highlight_color = WD_COLOR_INDEX.YELLOW
                run.font.name = 'Arial'
                run.font.size = Pt(12)

            last_end = end

        after = original_text[last_end:]
        if after:
            run = para.add_run(after)
            run.font.name = 'Arial'
            run.font.size = Pt(12)
print("66666")
def process_docx(path, sql_id):
    doc = Document(path)
    conn = get_db_connection()
    cursor = conn.cursor()
    variables_not_found = []
    counters = {}

    # שליפת כל הערכים מ-Mongo לפי sql_id
    mongo_data = {}
    for field in values_collection.find():
        # בדיקה אם יש שדה sql_id לפני הוספת הנתונים
        if 'sql_id' in field:
            mongo_data[field['sql_id']] = field.get('value')
        else:
            print(f"⏩ דילוג על רשומה ללא sql_id: {field}")

    print("123")

    # גוף
    process_paragraphs(doc.paragraphs, cursor, mongo_data, counters, variables_not_found, sql_id)

    # טבלאות
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                process_paragraphs(cell.paragraphs, cursor, mongo_data, counters, variables_not_found, sql_id)

    # כותרות עליונות ותחתונות
    for section in doc.sections:
        process_paragraphs(section.header.paragraphs, cursor, mongo_data, counters, variables_not_found, sql_id)
        process_paragraphs(section.footer.paragraphs, cursor, mongo_data, counters, variables_not_found, sql_id)

    # שמירה
    dir_path = os.path.dirname(path)
    filename_wo_ext = os.path.splitext(os.path.basename(path))[0]
    output_path = os.path.join(dir_path, f"{filename_wo_ext} טופל.docx")
    doc.save(output_path)

    print("✅ קובץ נשמר כ:", output_path)
    if variables_not_found:
        print("⚠️ משתנים שלא נמצאו:", variables_not_found)


# ---------- הרצה ----------

input_path = "C:\\Users\\User\\Desktop\\מסמכים לפרויקט\\ממופה\\יפוי כוח.doc"
if input_path.endswith(".doc"):
    input_path = convert_doc_to_docx(input_path)
process_docx(input_path,"3")

11111
22222
33333
66666
⏩ דילוג על רשומה ללא sql_id: {'_id': ObjectId('68073e0127e131bc0c90880e'), 'buyer_first_name': 'yossi'}
⏩ דילוג על רשומה ללא sql_id: {'_id': ObjectId('68077ece27e131bc0c90880f'), 'customer_id': '10', 'buyer_first_name': 'yehudit'}
123
44444
44444
44444
44444
44444
✅ קובץ נשמר כ: C:\Users\User\Desktop\מסמכים לפרויקט\ממופה\יפוי כוח טופל.docx
⚠️ משתנים שלא נמצאו: ['3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3']


In [101]:
from docx import Document
import re
from collections import defaultdict

# הנתיב המעודכן לקובץ ה- .docx
file_path = "C:\\Users\\User\\Desktop\\מסמכים לפרויקט\\ממופה\\יפוי כוח.docx"

# קריאה של קובץ וורד
doc = Document(file_path)

# פונקציה לזיהוי משתנים (שדות ממופים) מהקובץ
def extract_variables_from_paragraph(paragraph):
    pattern = r"{{(.*?)}}"
    matches = re.findall(pattern, paragraph.text)
    return matches

# מילון לאחסון משתנים לפי המילה הראשונה של המשתנה
variables = defaultdict(list)

# קיבוץ משתנים לפי המילה הראשונה של המשתנה
def group_variables_by_name():
    for paragraph in doc.paragraphs:
        variables_in_paragraph = extract_variables_from_paragraph(paragraph)
        for var in variables_in_paragraph:
            name, var_type, length, required, description = var.split(',')
            first_word = name.split('_')[0]  # לוקחים את המילה הראשונה בשם המשתנה (לפי חיבור בין מילים עם _)
            variables[first_word].append(f"{{{{{name.strip()},{var_type.strip()},{length.strip()},{required.strip()},{description.strip()}}}}}")

# שלב זיהוי המשתנים וקטגוריותיהם
group_variables_by_name()

# הצגת המשתנים בקבוצות לפי המילה הראשונה של המשתנה
for category, var_list in variables.items():
    print(f"קטגוריה: {category}")
    print(" ".join(var_list))


קטגוריה: buyer
{{buyer_first_name,varchar(50),50,*,שם פרטי קונה}} {{buyer_last_name,varchar(50),50,*,שם משפחה קונה}} {{buyer_id,varchar(9),9,*,מספר זהות קונה}}
קטגוריה: property
{{property_address_street,varchar(50),50,*,כתובת הנכס – רחוב}} {{property_address_number,varchar(10),10,*,כתובת הנכס – מספר בנין}} {{property_address_city,varchar(20),20,*,כתובת הנכס – עיר}} {{property_block_number,varchar(10),10,*,גוש}} {{property_parcel_number,varchar(10),10,*,חלקה}} {{property_subparcel_numbers,varchar(10),10,*,תת-חלקה}}
קטגוריה: seller
{{seller_first_name,varchar(50),50,*,שם פרטי מוכר}} {{seller_last_name,varchar(50),50,*,שם משפחה מוכר}} {{seller_id,varchar(9),9,*,מספר זהות מוכר}}
